In [ ]:
# Install required packages
!pip install kagglehub tensorflow pillow matplotlib seaborn scikit-learn

# Import libraries
import kagglehub
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import random
from google.colab import files
import io
import gc

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("TensorFlow version:", tf.__version__)
print("✅ Dependencies installed and imported successfully!")

In [ ]:
# Download the dataset
print("Downloading dataset...")
path = kagglehub.dataset_download("alxmamaev/flowers-recognition")
print("Path to dataset files:", path)

# Define flower classes and their scientific names
FLOWER_CLASSES = {
    'daisy': 'Bellis perennis',
    'dandelion': 'Taraxacum officinale',
    'rose': 'Rosa',
    'sunflower': 'Helianthus annuus',
    'tulip': 'Tulipa'
}

CLASS_NAMES = list(FLOWER_CLASSES.keys())
print("Flower classes:", CLASS_NAMES)
print("✅ Dataset downloaded and classes defined!")

In [ ]:
def load_data_batch(data_path, class_name, max_images=200, img_size=(224, 224)):
    """Load images for a single class to save memory"""
    images = []
    labels = []
    
    flowers_path = os.path.join(data_path, 'flowers')
    class_path = os.path.join(flowers_path, class_name)
    class_idx = CLASS_NAMES.index(class_name)
    
    if os.path.exists(class_path):
        print(f"Loading {class_name} images...")
        image_files = [f for f in os.listdir(class_path) 
                      if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        
        # Shuffle and limit images to prevent memory issues
        random.shuffle(image_files)
        image_files = image_files[:max_images]
        
        for img_file in image_files:
            img_path = os.path.join(class_path, img_file)
            try:
                # Load and resize image
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(img, img_size)
                    img = img / 255.0  # Normalize
                    
                    images.append(img)
                    labels.append(class_idx)
            except Exception as e:
                continue
        
        print(f"Loaded {len(images)} {class_name} images")
    
    return np.array(images), np.array(labels)

# Load data class by class to manage memory
print("Loading data in batches to manage memory...")
all_images = []
all_labels = []

for class_name in CLASS_NAMES:
    images, labels = load_data_batch(path, class_name, max_images=200)
    all_images.append(images)
    all_labels.append(labels)
    # Force garbage collection
    gc.collect()

# Combine all data
X = np.concatenate(all_images, axis=0)
y = np.concatenate(all_labels, axis=0)

print(f"Total images loaded: {len(X)}")
print(f"Image shape: {X[0].shape}")

# Clear intermediate variables to save memory
del all_images, all_labels
gc.collect()

print("✅ Data loaded successfully!")


In [ ]:
# Split the data
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

print(f"Training set: {len(X_train)} images")
print(f"Validation set: {len(X_val)} images")
print(f"Test set: {len(X_test)} images")

# Clear original arrays to save memory
del X, y, X_temp, y_temp
gc.collect()

print("✅ Data split completed!")

In [ ]:
def plot_sample_images():
    """Plot sample images from each class"""
    fig, axes = plt.subplots(2, 3, figsize=(15, 8))
    axes = axes.ravel()
    
    for i, class_name in enumerate(CLASS_NAMES):
        # Find first image of this class
        class_indices = np.where(y_train == i)[0]
        if len(class_indices) > 0:
            sample_idx = class_indices[0]
            axes[i].imshow(X_train[sample_idx])
            axes[i].set_title(f"{class_name.title()}\n({FLOWER_CLASSES[class_name]})")
            axes[i].axis('off')
    
    # Plot class distribution
    unique, counts = np.unique(y_train, return_counts=True)
    class_names_for_plot = [CLASS_NAMES[i] for i in unique]
    
    axes[5].bar(class_names_for_plot, counts)
    axes[5].set_title('Class Distribution')
    axes[5].set_ylabel('Number of Images')
    plt.setp(axes[5].get_xticklabels(), rotation=45)
    
    plt.tight_layout()
    plt.show()

plot_sample_images()
print("✅ Data visualization completed!")

In [ ]:
def create_lightweight_model():
    """Create a lightweight CNN model for flower classification"""
    model = keras.Sequential([
        # Data augmentation
        layers.RandomFlip('horizontal'),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
        
        # Lightweight convolutional layers
        layers.Conv2D(32, 3, activation='relu', input_shape=(224, 224, 3)),
        layers.MaxPooling2D(2),
        layers.Dropout(0.25),
        
        layers.Conv2D(64, 3, activation='relu'),
        layers.MaxPooling2D(2),
        layers.Dropout(0.25),
        
        layers.Conv2D(128, 3, activation='relu'),
        layers.MaxPooling2D(2),
        layers.Dropout(0.25),
        
        # Global average pooling instead of flatten to reduce parameters
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(len(CLASS_NAMES), activation='softmax')
    ])
    
    return model

# Create and compile the model
model = create_lightweight_model()
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("Model architecture:")
model.summary()
print("✅ Model created successfully!")

In [ ]:
# Training callbacks
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=8,
        restore_best_weights=True,
        verbose=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_accuracy',
        factor=0.5,
        patience=4,
        min_lr=1e-6,
        verbose=1
    )
]

print("✅ Training callbacks configured!")


In [ ]:
# Train the model
print("Starting training...")
history = model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=16,  # Smaller batch size
    validation_data=(X_val, y_val),
    callbacks=callbacks,
    verbose=1
)

print("✅ Training completed!")


In [ ]:
def plot_training_history(history):
    """Plot training and validation accuracy/loss"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    # Accuracy plot
    ax1.plot(history.history['accuracy'], label='Training Accuracy')
    ax1.plot(history.history['val_accuracy'], label='Validation Accuracy')
    ax1.set_title('Model Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    ax1.grid(True)
    
    # Loss plot
    ax2.plot(history.history['loss'], label='Training Loss')
    ax2.plot(history.history['val_loss'], label='Validation Loss')
    ax2.set_title('Model Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()

plot_training_history(history)
print("✅ Training visualization completed!")


In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

# Generate predictions for test set (in batches to manage memory)
def predict_in_batches(model, X_test, batch_size=32):
    """Predict in batches to manage memory"""
    predictions = []
    for i in range(0, len(X_test), batch_size):
        batch = X_test[i:i+batch_size]
        pred_batch = model.predict(batch, verbose=0)
        predictions.extend(pred_batch)
    return np.array(predictions)

y_pred = predict_in_batches(model, X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes, target_names=CLASS_NAMES))

print("✅ Model evaluation completed!")

In [ ]:
def plot_confusion_matrix():
    """Plot confusion matrix"""
    cm = confusion_matrix(y_test, y_pred_classes)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=CLASS_NAMES, yticklabels=CLASS_NAMES)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

plot_confusion_matrix()
print("✅ Confusion matrix plotted!")

In [ ]:
def predict_flower(image_path_or_array, model, display_image=True):
    """Predict flower class from image"""
    try:
        # Handle different input types
        if isinstance(image_path_or_array, str):
            img = cv2.imread(image_path_or_array)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        elif isinstance(image_path_or_array, np.ndarray):
            img = image_path_or_array
        else:
            img = np.array(image_path_or_array)
        
        # Preprocess image
        original_img = img.copy()
        img_resized = cv2.resize(img, (224, 224))
        img_normalized = img_resized / 255.0
        img_batch = np.expand_dims(img_normalized, axis=0)
        
        # Make prediction
        predictions = model.predict(img_batch, verbose=0)
        predicted_class_idx = np.argmax(predictions[0])
        confidence = predictions[0][predicted_class_idx]
        
        # Get class name and scientific name
        predicted_class = CLASS_NAMES[predicted_class_idx]
        scientific_name = FLOWER_CLASSES[predicted_class]
        
        # Display results
        if display_image:
            plt.figure(figsize=(12, 5))
            
            # Original image
            plt.subplot(1, 2, 1)
            plt.imshow(original_img)
            plt.title('Input Image')
            plt.axis('off')
            
            # Prediction probabilities
            plt.subplot(1, 2, 2)
            y_pos = np.arange(len(CLASS_NAMES))
            colors = ['green' if i == predicted_class_idx else 'lightblue' 
                     for i in range(len(CLASS_NAMES))]
            plt.barh(y_pos, predictions[0], color=colors)
            plt.yticks(y_pos, [name.title() for name in CLASS_NAMES])
            plt.xlabel('Confidence')
            plt.title('Prediction Probabilities')
            plt.tight_layout()
            plt.show()
        
        print(f"\n🌸 Prediction Results:")
        print(f"Common Name: {predicted_class.title()}")
        print(f"Scientific Name: {scientific_name}")
        print(f"Confidence: {confidence:.2%}")
        
        return predicted_class, scientific_name, confidence
        
    except Exception as e:
        print(f"Error in prediction: {e}")
        return None, None, None

def get_flower_info(flower_name):
    """Get additional information about the flower"""
    flower_info = {
        'daisy': {
            'family': 'Asteraceae',
            'description': 'Small white flowers with yellow centers, commonly found in lawns and meadows.',
            'season': 'Spring to Fall',
            'habitat': 'Grasslands, meadows, lawns'
        },
        'dandelion': {
            'family': 'Asteraceae',
            'description': 'Bright yellow composite flowers that turn into white seed heads.',
            'season': 'Spring to Fall',
            'habitat': 'Disturbed soils, lawns, roadsides'
        },
        'rose': {
            'family': 'Rosaceae',
            'description': 'Fragrant flowers with layered petals, often thorny stems.',
            'season': 'Spring to Fall',
            'habitat': 'Gardens, wild areas with good drainage'
        },
        'sunflower': {
            'family': 'Asteraceae',
            'description': 'Large yellow flowers that follow the sun, with edible seeds.',
            'season': 'Summer to Fall',
            'habitat': 'Fields, gardens, sunny areas'
        },
        'tulip': {
            'family': 'Liliaceae',
            'description': 'Cup-shaped flowers in various colors, blooming from bulbs.',
            'season': 'Spring',
            'habitat': 'Gardens, parks, temperate regions'
        }
    }
    return flower_info.get(flower_name.lower(), {})

print("✅ Prediction functions defined!")


In [ ]:
def test_sample_predictions(num_samples=3):
    """Test predictions on random samples from test set"""
    print("Testing predictions on random samples from test set:")
    
    random_indices = random.sample(range(len(X_test)), num_samples)
    
    for i, idx in enumerate(random_indices):
        print(f"\n--- Sample {i+1} ---")
        true_label = CLASS_NAMES[y_test[idx]]
        print(f"True label: {true_label.title()} ({FLOWER_CLASSES[true_label]})")
        
        predicted_class, scientific_name, confidence = predict_flower(
            X_test[idx], model, display_image=True
        )
        
        # Show additional info
        flower_info = get_flower_info(predicted_class)
        if flower_info:
            print(f"Family: {flower_info.get('family', 'N/A')}")
            print(f"Description: {flower_info.get('description', 'N/A')}")

test_sample_predictions(2)
print("✅ Sample predictions completed!")


In [ ]:
def upload_and_predict():
    """Upload an image and get prediction with detailed information"""
    print("Upload an image to classify:")
    uploaded = files.upload()
    
    for filename in uploaded.keys():
        print(f"\nProcessing {filename}...")
        
        # Read the uploaded file
        image_data = uploaded[filename]
        image = Image.open(io.BytesIO(image_data))
        image_array = np.array(image)
        
        # Make prediction
        predicted_class, scientific_name, confidence = predict_flower(
            image_array, model, display_image=True
        )
        
        if predicted_class:
            # Show additional flower information
            flower_info = get_flower_info(predicted_class)
            if flower_info:
                print(f"\n📚 Additional Information:")
                print(f"Family: {flower_info.get('family', 'N/A')}")
                print(f"Description: {flower_info.get('description', 'N/A')}")
                print(f"Blooming Season: {flower_info.get('season', 'N/A')}")
                print(f"Typical Habitat: {flower_info.get('habitat', 'N/A')}")

print("✅ Interactive upload function ready!")


In [ ]:
model.save('flower_classifier_model.h5')
print("✅ Model saved as 'flower_classifier_model.h5'")

print("\n" + "="*60)
print("🌻 FLOWER CLASSIFICATION PROJECT COMPLETED! 🌻")
print("="*60)
print("\n📋 Usage Instructions:")
print("1. To classify uploaded images: upload_and_predict()")
print("2. To test with random samples: test_sample_predictions(3)")
print("3. To predict from array: predict_flower(image_array, model)")
print("\n🌸 Supported Flowers:")
for flower, scientific in FLOWER_CLASSES.items():
    print(f"   • {flower.title()} ({scientific})")
print("\n💡 Tips:")
print("   • Use clear, well-lit flower images for best results")
print("   • The model works best with single flower images")
print("   • Confidence scores above 70% are generally reliable")
print("="*60)

# Memory cleanup
gc.collect()
print("✅ Memory cleaned up!")